# SDTR data preparation and training

```
created by: Muhammad Sulton Tauhid
sources: 
    1. https://itnext.io/nlp-named-entity-recognition-ner-with-spacy-and-python-dabaf843cab2
    2. https://stackoverflow.com/questions/56642816/valueerror-e024-could-not-find-an-optimal-move-to-supervise-the-parser
```

# STEP 1 - Data Preparation

In [1]:
import json

In [2]:
labeled_data_ajb_1 = []     # data preparation for AJB document page one
labeled_data_ajb_23 = []    # data preparation for AJB document page two
training_data_ajb_1 = []    # data convertion from doccano into spacy variable for AJB document page one
training_data_ajb_23 = []   # data convertion from doccano into spacy variable for AJB document page two

In [3]:
# read AJB document page one file
with open(r"../../dataset/export/ajb_1.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data_ajb_1.append(data)

# read AJB document page two and three file
with open(r"../../dataset/export/ajb_23.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data_ajb_23.append(data)

In [4]:
# convert AJB document page one labeled_data doccano version into training_data spacy version
for entry in labeled_data_ajb_1:
    entities = []
    for e in entry['label']:
        entities.append((e[0], e[1], e[2]))
    spacy_entry = (entry['data'], { "entities": entities })
    training_data_ajb_1.append(spacy_entry)
    
# convert AJB document page two and three labeled_data doccano version into training_data spacy version
for entry in labeled_data_ajb_23:
    entities = []
    for e in entry['label']:
        entities.append((e[0], e[1], e[2]))
    spacy_entry = (entry['data'], { "entities": entities })
    training_data_ajb_23.append(spacy_entry)

In [5]:
training_data_ajb_1

[('AJB1_1988_R_1\tmengenai jual bali ml ik adat tanah hak nomor c3603 akta jualbeli no1382jragr 19 88 pada hari ini hari selaga tanggal 15 nopamber 1988 datang menghadap kepada kami camat kepala wilayah kecamatan gapond0 h olementert dan negei uengan surarkeputusamya berdasarkan ketentuan dalam pasal 5 peraturan menteri tangga agraria no 101961 bertindak sebagai m m lthi han afig ra nomor dije penjabat pembuat akta tanah yang dimaksudkan dalam pasal 19 peraturan pemerintah no 10 tahun 1961 tentang pendaftaran tanah untuk wilayah kebupaten daerah tinskatttnearane dengan dihadliri oleh saksisaksi yang kami kenaldiperkenalkan kepada kami dan akan dise butkan dibagian akhir akta ini i naa u m ur ny tinaw ati 48 tahun 1tbu rumattanggam indonesia aesoleh al1 rt004014kelura han sukasari kecamatan tangerang kota administratif tangerang pakerjaan kownhemegarwn aa at selanjutnya disebut peninal 1 jaka pihrkasa n a a a ii u m u 19tal 19 tahun pelada i indonosia 1 j1mtharypno rt00209 kelurahan suk

# STEP 2 - Create the model for AJB document page one 

In [6]:
import spacy
import random
import datetime
from spacy.training.example import Example
from spacy.scorer import Scorer

In [7]:
# create the spacy pipeline and define the label
nlp = spacy.blank("id")
ner = nlp.add_pipe('ner')

ner.add_label("no_ajb")
ner.add_label("penjual_nama")
ner.add_label("penjual_umur")
ner.add_label("penjual_pekerjaan")
ner.add_label("penjual_agama")
ner.add_label("penjual_kewarganegaraan")
ner.add_label("penjual_alamat")
ner.add_label("pembeli_nama")
ner.add_label("pembeli_umur")
ner.add_label("pembeli_pekerjaan")
ner.add_label("pembeli_agama")
ner.add_label("pembeli_kewarganegaraan")
ner.add_label("pembeli_alamat")

print(ner.move_names)

['B-no_ajb', 'I-no_ajb', 'L-no_ajb', 'U-no_ajb', 'B-penjual_nama', 'I-penjual_nama', 'L-penjual_nama', 'U-penjual_nama', 'B-penjual_umur', 'I-penjual_umur', 'L-penjual_umur', 'U-penjual_umur', 'B-penjual_pekerjaan', 'I-penjual_pekerjaan', 'L-penjual_pekerjaan', 'U-penjual_pekerjaan', 'B-penjual_agama', 'I-penjual_agama', 'L-penjual_agama', 'U-penjual_agama', 'B-penjual_kewarganegaraan', 'I-penjual_kewarganegaraan', 'L-penjual_kewarganegaraan', 'U-penjual_kewarganegaraan', 'B-penjual_alamat', 'I-penjual_alamat', 'L-penjual_alamat', 'U-penjual_alamat', 'B-pembeli_nama', 'I-pembeli_nama', 'L-pembeli_nama', 'U-pembeli_nama', 'B-pembeli_umur', 'I-pembeli_umur', 'L-pembeli_umur', 'U-pembeli_umur', 'B-pembeli_pekerjaan', 'I-pembeli_pekerjaan', 'L-pembeli_pekerjaan', 'U-pembeli_pekerjaan', 'B-pembeli_agama', 'I-pembeli_agama', 'L-pembeli_agama', 'U-pembeli_agama', 'B-pembeli_kewarganegaraan', 'I-pembeli_kewarganegaraan', 'L-pembeli_kewarganegaraan', 'U-pembeli_kewarganegaraan', 'B-pembeli_alam

In [9]:
# initialize the training
nlp.begin_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

epochs = 1000
print("current date: {0} {1} {2}".format(datetime.datetime.now().day, datetime.datetime.now().strftime("%B"), datetime.datetime.now().year))
with nlp.disable_pipes(*other_pipes): # only train NER    
    for i in range(epochs):
        random.shuffle(training_data_ajb_1)
        losses = {}

        for batch in spacy.util.minibatch(training_data_ajb_1, size=2):
            for text, entities in batch:
                try:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, entities)

                    nlp.update([example], losses=losses, drop=0.3)

                except Exception as error:
                    continue
        print("iteration: {0} | loss: {1} | time: {2}".format(i + 1, losses, datetime.datetime.now().strftime("%H:%M:%S")))
    
# save the result of training into disk
nlp.to_disk("ajb_1_model")

current date: 14 July 2021
iteration: 1 | loss: {'ner': 2340.8085682220635} | time: 07:29:02
iteration: 2 | loss: {'ner': 939.2359305594395} | time: 07:29:03
iteration: 3 | loss: {'ner': 874.415960279567} | time: 07:29:05
iteration: 4 | loss: {'ner': 1051.126359122598} | time: 07:29:06
iteration: 5 | loss: {'ner': 975.1124564438279} | time: 07:29:08
iteration: 6 | loss: {'ner': 874.3878868162317} | time: 07:29:09
iteration: 7 | loss: {'ner': 860.7096612305664} | time: 07:29:11
iteration: 8 | loss: {'ner': 1211.350632241985} | time: 07:29:12
iteration: 9 | loss: {'ner': 781.7754158113496} | time: 07:29:14
iteration: 10 | loss: {'ner': 778.7774252547281} | time: 07:29:16
iteration: 11 | loss: {'ner': 828.2216454443763} | time: 07:29:17
iteration: 12 | loss: {'ner': 771.1044266044184} | time: 07:29:19
iteration: 13 | loss: {'ner': 933.9940385797569} | time: 07:29:20
iteration: 14 | loss: {'ner': 803.0229083239723} | time: 07:29:22
iteration: 15 | loss: {'ner': 661.3731078043451} | time: 0

iteration: 124 | loss: {'ner': 16.362294496430124} | time: 07:32:22
iteration: 125 | loss: {'ner': 16.0204033916155} | time: 07:32:24
iteration: 126 | loss: {'ner': 16.017181208884818} | time: 07:32:25
iteration: 127 | loss: {'ner': 19.63772382060771} | time: 07:32:27
iteration: 128 | loss: {'ner': 28.804708979153} | time: 07:32:29
iteration: 129 | loss: {'ner': 14.99807061114872} | time: 07:32:30
iteration: 130 | loss: {'ner': 17.57256763812293} | time: 07:32:32
iteration: 131 | loss: {'ner': 12.844229343266706} | time: 07:32:34
iteration: 132 | loss: {'ner': 17.202324316938682} | time: 07:32:35
iteration: 133 | loss: {'ner': 20.646290481270196} | time: 07:32:37
iteration: 134 | loss: {'ner': 68.6629891213841} | time: 07:32:38
iteration: 135 | loss: {'ner': 38.92291788677649} | time: 07:32:40
iteration: 136 | loss: {'ner': 16.503271328843283} | time: 07:32:42
iteration: 137 | loss: {'ner': 21.01421032047646} | time: 07:32:43
iteration: 138 | loss: {'ner': 15.831551667960996} | time: 0

iteration: 246 | loss: {'ner': 10.97324596816161} | time: 07:35:40
iteration: 247 | loss: {'ner': 18.2735164931044} | time: 07:35:41
iteration: 248 | loss: {'ner': 41.88392099178834} | time: 07:35:43
iteration: 249 | loss: {'ner': 21.715803939099594} | time: 07:35:44
iteration: 250 | loss: {'ner': 12.525585687814674} | time: 07:35:46
iteration: 251 | loss: {'ner': 12.361600196390778} | time: 07:35:48
iteration: 252 | loss: {'ner': 19.70012090207201} | time: 07:35:50
iteration: 253 | loss: {'ner': 18.689078707688157} | time: 07:35:51
iteration: 254 | loss: {'ner': 30.15995725068718} | time: 07:35:53
iteration: 255 | loss: {'ner': 17.908218825658928} | time: 07:35:54
iteration: 256 | loss: {'ner': 10.161614223550643} | time: 07:35:56
iteration: 257 | loss: {'ner': 7.501644723758193} | time: 07:35:58
iteration: 258 | loss: {'ner': 24.975367554898018} | time: 07:35:59
iteration: 259 | loss: {'ner': 11.491014586407745} | time: 07:36:01
iteration: 260 | loss: {'ner': 4.250477796070238} | tim

iteration: 368 | loss: {'ner': 11.36156210211605} | time: 07:38:55
iteration: 369 | loss: {'ner': 12.746065163984209} | time: 07:38:57
iteration: 370 | loss: {'ner': 8.961073134998456} | time: 07:38:58
iteration: 371 | loss: {'ner': 11.030166587351268} | time: 07:39:00
iteration: 372 | loss: {'ner': 10.562591822920593} | time: 07:39:01
iteration: 373 | loss: {'ner': 10.721464776081326} | time: 07:39:03
iteration: 374 | loss: {'ner': 13.547617543978161} | time: 07:39:05
iteration: 375 | loss: {'ner': 6.241316871604111} | time: 07:39:06
iteration: 376 | loss: {'ner': 17.995210838471632} | time: 07:39:08
iteration: 377 | loss: {'ner': 22.439619572226725} | time: 07:39:10
iteration: 378 | loss: {'ner': 7.609341974793471} | time: 07:39:11
iteration: 379 | loss: {'ner': 5.597067437488461} | time: 07:39:13
iteration: 380 | loss: {'ner': 12.204614260334363} | time: 07:39:14
iteration: 381 | loss: {'ner': 17.561624814990697} | time: 07:39:16
iteration: 382 | loss: {'ner': 10.788016193878171} | 

iteration: 490 | loss: {'ner': 8.812362433422548} | time: 07:42:15
iteration: 491 | loss: {'ner': 8.681184583936979} | time: 07:42:17
iteration: 492 | loss: {'ner': 9.597721679604938} | time: 07:42:18
iteration: 493 | loss: {'ner': 9.50471308797037} | time: 07:42:20
iteration: 494 | loss: {'ner': 6.686004617657861} | time: 07:42:21
iteration: 495 | loss: {'ner': 11.178767386280132} | time: 07:42:23
iteration: 496 | loss: {'ner': 10.193550313726332} | time: 07:42:25
iteration: 497 | loss: {'ner': 11.114547127881375} | time: 07:42:26
iteration: 498 | loss: {'ner': 1.7809664629515092} | time: 07:42:28
iteration: 499 | loss: {'ner': 20.02131302930173} | time: 07:42:29
iteration: 500 | loss: {'ner': 10.359310097529985} | time: 07:42:31
iteration: 501 | loss: {'ner': 12.270353242644676} | time: 07:42:32
iteration: 502 | loss: {'ner': 4.747402339841601} | time: 07:42:34
iteration: 503 | loss: {'ner': 8.689755892588309} | time: 07:42:36
iteration: 504 | loss: {'ner': 10.905068042650958} | time

iteration: 612 | loss: {'ner': 5.266293128832719} | time: 07:45:32
iteration: 613 | loss: {'ner': 9.3093376849076} | time: 07:45:33
iteration: 614 | loss: {'ner': 8.615292013738701} | time: 07:45:35
iteration: 615 | loss: {'ner': 10.712043776403796} | time: 07:45:36
iteration: 616 | loss: {'ner': 12.948089421854776} | time: 07:45:38
iteration: 617 | loss: {'ner': 11.631617907894917} | time: 07:45:40
iteration: 618 | loss: {'ner': 4.348281103831491} | time: 07:45:41
iteration: 619 | loss: {'ner': 3.205785208444821} | time: 07:45:43
iteration: 620 | loss: {'ner': 36.852414972112} | time: 07:45:45
iteration: 621 | loss: {'ner': 9.97865557740902} | time: 07:45:46
iteration: 622 | loss: {'ner': 8.927442327164107} | time: 07:45:48
iteration: 623 | loss: {'ner': 11.963474199658277} | time: 07:45:49
iteration: 624 | loss: {'ner': 6.82700059687377} | time: 07:45:51
iteration: 625 | loss: {'ner': 11.856722506824479} | time: 07:45:52
iteration: 626 | loss: {'ner': 8.920172361515286} | time: 07:45

iteration: 734 | loss: {'ner': 4.211664996938128} | time: 07:48:55
iteration: 735 | loss: {'ner': 9.479368553787069} | time: 07:48:56
iteration: 736 | loss: {'ner': 6.965345026007202} | time: 07:48:58
iteration: 737 | loss: {'ner': 8.707220317291375} | time: 07:49:00
iteration: 738 | loss: {'ner': 3.5868099867281518} | time: 07:49:01
iteration: 739 | loss: {'ner': 6.93975598120145} | time: 07:49:03
iteration: 740 | loss: {'ner': 7.0660884435233955} | time: 07:49:05
iteration: 741 | loss: {'ner': 18.76861174209034} | time: 07:49:06
iteration: 742 | loss: {'ner': 13.409604250975882} | time: 07:49:08
iteration: 743 | loss: {'ner': 4.49451484401366} | time: 07:49:09
iteration: 744 | loss: {'ner': 12.217321525397406} | time: 07:49:11
iteration: 745 | loss: {'ner': 10.056333382324} | time: 07:49:13
iteration: 746 | loss: {'ner': 6.488511658180439} | time: 07:49:14
iteration: 747 | loss: {'ner': 16.50975208429465} | time: 07:49:16
iteration: 748 | loss: {'ner': 5.103219563244738} | time: 07:4

iteration: 856 | loss: {'ner': 6.098514377284938} | time: 07:52:15
iteration: 857 | loss: {'ner': 7.073840728340092} | time: 07:52:17
iteration: 858 | loss: {'ner': 14.880297245770885} | time: 07:52:18
iteration: 859 | loss: {'ner': 6.488615806112637} | time: 07:52:20
iteration: 860 | loss: {'ner': 8.186647738816946} | time: 07:52:22
iteration: 861 | loss: {'ner': 4.425374636326672} | time: 07:52:23
iteration: 862 | loss: {'ner': 8.349812166551247} | time: 07:52:25
iteration: 863 | loss: {'ner': 7.441766089727832} | time: 07:52:27
iteration: 864 | loss: {'ner': 7.843971959315627} | time: 07:52:29
iteration: 865 | loss: {'ner': 5.108383573895645} | time: 07:52:30
iteration: 866 | loss: {'ner': 17.734744601252856} | time: 07:52:32
iteration: 867 | loss: {'ner': 10.30059223514044} | time: 07:52:33
iteration: 868 | loss: {'ner': 22.12804711223767} | time: 07:52:35
iteration: 869 | loss: {'ner': 6.179090705451061} | time: 07:52:37
iteration: 870 | loss: {'ner': 1.7124587975549204} | time: 0

iteration: 978 | loss: {'ner': 4.470754756361765} | time: 07:55:38
iteration: 979 | loss: {'ner': 10.194426970935668} | time: 07:55:40
iteration: 980 | loss: {'ner': 8.333023341609646} | time: 07:55:41
iteration: 981 | loss: {'ner': 17.72949910364147} | time: 07:55:43
iteration: 982 | loss: {'ner': 11.83446055192091} | time: 07:55:45
iteration: 983 | loss: {'ner': 10.656130267699991} | time: 07:55:46
iteration: 984 | loss: {'ner': 10.725759701745286} | time: 07:55:48
iteration: 985 | loss: {'ner': 9.259270393095512} | time: 07:55:50
iteration: 986 | loss: {'ner': 5.3800171962243395} | time: 07:55:51
iteration: 987 | loss: {'ner': 10.23025064905097} | time: 07:55:53
iteration: 988 | loss: {'ner': 6.367829411415846} | time: 07:55:55
iteration: 989 | loss: {'ner': 4.4040589163110475} | time: 07:55:56
iteration: 990 | loss: {'ner': 4.046008989355179} | time: 07:55:58
iteration: 991 | loss: {'ner': 4.671796452236133} | time: 07:56:00
iteration: 992 | loss: {'ner': 3.5408302078278977} | time

# STEP 3 - Create the model for AJB document page two and three 

In [10]:
# create the spacy pipeline and define the label
nlp_new = spacy.blank("id")
ner_new = nlp_new.add_pipe('ner')

ner_new.add_label("no_kohir_girik")
ner_new.add_label("no_persil")
ner_new.add_label("provinsi_tanah")
ner_new.add_label("kabupaten_tanah")
ner_new.add_label("kecamatan_tanah")
ner_new.add_label("kelurahan_tanah")
ner_new.add_label("jalan_tanah")
ner_new.add_label("harga_tanah")
ner_new.add_label("luas_tanah")

print(ner_new.move_names)

['B-no_kohir_girik', 'I-no_kohir_girik', 'L-no_kohir_girik', 'U-no_kohir_girik', 'B-no_persil', 'I-no_persil', 'L-no_persil', 'U-no_persil', 'B-provinsi_tanah', 'I-provinsi_tanah', 'L-provinsi_tanah', 'U-provinsi_tanah', 'B-kabupaten_tanah', 'I-kabupaten_tanah', 'L-kabupaten_tanah', 'U-kabupaten_tanah', 'B-kecamatan_tanah', 'I-kecamatan_tanah', 'L-kecamatan_tanah', 'U-kecamatan_tanah', 'B-kelurahan_tanah', 'I-kelurahan_tanah', 'L-kelurahan_tanah', 'U-kelurahan_tanah', 'B-jalan_tanah', 'I-jalan_tanah', 'L-jalan_tanah', 'U-jalan_tanah', 'B-harga_tanah', 'I-harga_tanah', 'L-harga_tanah', 'U-harga_tanah', 'B-luas_tanah', 'I-luas_tanah', 'L-luas_tanah', 'U-luas_tanah']


In [11]:
# initialize the training
nlp_new.begin_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

epochs = 1000
print("current date: {0} {1} {2}".format(datetime.datetime.now().day, datetime.datetime.now().strftime("%B"), datetime.datetime.now().year))
with nlp_new.disable_pipes(*other_pipes): # only train NER    
    for i in range(epochs):
        random.shuffle(training_data_ajb_23)
        losses = {}

        for batch in spacy.util.minibatch(training_data_ajb_23, size=2):
            for text, entities in batch:
                try:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, entities)

                    nlp_new.update([example], losses=losses, drop=0.3)
                except Exception as error:
                    continue
                
        print("iteration: {0} | loss: {1} | time: {2}".format(i + 1, losses, datetime.datetime.now().strftime("%H:%M:%S")))
    
# save the result of training into disk
nlp_new.to_disk("ajb_23_model")

current date: 14 July 2021


/home/msultont/Desktop/project-seighneur/SDTR/model/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "AJB2_2008_D_1	para penghadap dikenal oleh sayapeng..." with entities "[(909, 926, 'no_persil'), (926, 953, 'no_kohir_gir...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/msultont/Desktop/project-seighneur/SDTR/model/lib/python3.9/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "AJB2_2008_R_1	para penghadap dikenal oleh sayapeng..." with entities "[(909, 926, 'no_persil'), (926, 953, 'no_kohir_gir...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/msultont/Desktop/project-seighneur

iteration: 1 | loss: {'ner': 1759.801959323086} | time: 08:22:39
iteration: 2 | loss: {'ner': 420.3782295494847} | time: 08:22:40
iteration: 3 | loss: {'ner': 366.3602386547522} | time: 08:22:40
iteration: 4 | loss: {'ner': 252.42683783996478} | time: 08:22:41
iteration: 5 | loss: {'ner': 436.07710765209396} | time: 08:22:42
iteration: 6 | loss: {'ner': 505.3364638365642} | time: 08:22:43
iteration: 7 | loss: {'ner': 517.9035093283858} | time: 08:22:44
iteration: 8 | loss: {'ner': 527.3690463067196} | time: 08:22:45
iteration: 9 | loss: {'ner': 283.1231111269724} | time: 08:22:46
iteration: 10 | loss: {'ner': 304.0129693873379} | time: 08:22:47
iteration: 11 | loss: {'ner': 237.7132714974836} | time: 08:22:47
iteration: 12 | loss: {'ner': 596.0964926329342} | time: 08:22:48
iteration: 13 | loss: {'ner': 335.5583866066299} | time: 08:22:49
iteration: 14 | loss: {'ner': 178.36252435813498} | time: 08:22:50
iteration: 15 | loss: {'ner': 356.4296727011091} | time: 08:22:51
iteration: 16 | 

iteration: 125 | loss: {'ner': 5.003287789301216} | time: 08:24:32
iteration: 126 | loss: {'ner': 6.572641810762079} | time: 08:24:33
iteration: 127 | loss: {'ner': 14.282556257521394} | time: 08:24:34
iteration: 128 | loss: {'ner': 6.269419414389995} | time: 08:24:34
iteration: 129 | loss: {'ner': 13.37467804984123} | time: 08:24:35
iteration: 130 | loss: {'ner': 7.35152854164617} | time: 08:24:36
iteration: 131 | loss: {'ner': 13.89474184877236} | time: 08:24:37
iteration: 132 | loss: {'ner': 18.922233404760544} | time: 08:24:38
iteration: 133 | loss: {'ner': 6.732562302634507} | time: 08:24:39
iteration: 134 | loss: {'ner': 6.083575635815893} | time: 08:24:40
iteration: 135 | loss: {'ner': 14.198503814472373} | time: 08:24:41
iteration: 136 | loss: {'ner': 9.1013671383141} | time: 08:24:42
iteration: 137 | loss: {'ner': 7.663323901192238} | time: 08:24:43
iteration: 138 | loss: {'ner': 23.60935168788071} | time: 08:24:44
iteration: 139 | loss: {'ner': 13.215165763345968} | time: 08:

iteration: 247 | loss: {'ner': 3.3058125613708476} | time: 08:26:22
iteration: 248 | loss: {'ner': 3.773975853328572} | time: 08:26:22
iteration: 249 | loss: {'ner': 4.275688710150819} | time: 08:26:23
iteration: 250 | loss: {'ner': 1.9685925800071788} | time: 08:26:24
iteration: 251 | loss: {'ner': 5.1930051380865} | time: 08:26:25
iteration: 252 | loss: {'ner': 7.857819778228016} | time: 08:26:26
iteration: 253 | loss: {'ner': 3.0681384279116553} | time: 08:26:27
iteration: 254 | loss: {'ner': 3.4646869219183505} | time: 08:26:28
iteration: 255 | loss: {'ner': 4.72488557767166} | time: 08:26:29
iteration: 256 | loss: {'ner': 5.180221541109976} | time: 08:26:30
iteration: 257 | loss: {'ner': 8.4296290909975} | time: 08:26:31
iteration: 258 | loss: {'ner': 4.398964899133975} | time: 08:26:31
iteration: 259 | loss: {'ner': 6.193102428602012} | time: 08:26:32
iteration: 260 | loss: {'ner': 4.005536988520152} | time: 08:26:33
iteration: 261 | loss: {'ner': 6.296112334924179} | time: 08:26

...

iteration: 491 | loss: {'ner': 2.1322816442335952} | time: 08:30:13
iteration: 492 | loss: {'ner': 3.065075359087055} | time: 08:30:14
iteration: 493 | loss: {'ner': 2.718886784311828} | time: 08:30:15
iteration: 494 | loss: {'ner': 2.117912086785227} | time: 08:30:16
iteration: 495 | loss: {'ner': 2.5031764887449235} | time: 08:30:17
iteration: 496 | loss: {'ner': 4.800018809694246} | time: 08:30:18
iteration: 497 | loss: {'ner': 2.002090506782472} | time: 08:30:19
iteration: 498 | loss: {'ner': 4.104074397626535} | time: 08:30:20
iteration: 499 | loss: {'ner': 2.3218880881717148} | time: 08:30:21
iteration: 500 | loss: {'ner': 2.0998006119569097} | time: 08:30:22
iteration: 501 | loss: {'ner': 2.0488572858473835} | time: 08:30:23
iteration: 502 | loss: {'ner': 2.8939139914818948} | time: 08:30:24
iteration: 503 | loss: {'ner': 1.7445554971951192} | time: 08:30:25
iteration: 504 | loss: {'ner': 4.002245692845695} | time: 08:30:26
iteration: 505 | loss: {'ner': 4.921440617688778} | tim

iteration: 613 | loss: {'ner': 3.042447494093963} | time: 08:32:17
iteration: 614 | loss: {'ner': 3.878680356341731} | time: 08:32:18
iteration: 615 | loss: {'ner': 3.758701249908099} | time: 08:32:19
iteration: 616 | loss: {'ner': 4.931544378986035} | time: 08:32:20
iteration: 617 | loss: {'ner': 4.938173935832995} | time: 08:32:21
iteration: 618 | loss: {'ner': 0.5199433382210389} | time: 08:32:22
iteration: 619 | loss: {'ner': 3.619677298893739} | time: 08:32:22
iteration: 620 | loss: {'ner': 1.9933577490220846} | time: 08:32:23
iteration: 621 | loss: {'ner': 2.9705465746260415} | time: 08:32:24
iteration: 622 | loss: {'ner': 3.0228308930976238} | time: 08:32:25
iteration: 623 | loss: {'ner': 6.004493473710947} | time: 08:32:26
iteration: 624 | loss: {'ner': 5.205703210936306} | time: 08:32:27
iteration: 625 | loss: {'ner': 2.43078296223015} | time: 08:32:28
iteration: 626 | loss: {'ner': 5.363291557882929} | time: 08:32:29
iteration: 627 | loss: {'ner': 1.4921803339843884} | time: 

iteration: 735 | loss: {'ner': 3.1105127386263107} | time: 08:34:11
iteration: 736 | loss: {'ner': 3.9615108910533228} | time: 08:34:12
iteration: 737 | loss: {'ner': 2.855854844449152} | time: 08:34:13
iteration: 738 | loss: {'ner': 2.1308143625750677} | time: 08:34:14
iteration: 739 | loss: {'ner': 12.300065484139795} | time: 08:34:15
iteration: 740 | loss: {'ner': 19.4927480997863} | time: 08:34:16
iteration: 741 | loss: {'ner': 11.021842916707932} | time: 08:34:17
iteration: 742 | loss: {'ner': 2.5040409962407377} | time: 08:34:18
iteration: 743 | loss: {'ner': 4.039434241790428} | time: 08:34:19
iteration: 744 | loss: {'ner': 2.5293042771870486} | time: 08:34:20
iteration: 745 | loss: {'ner': 2.000361673529413} | time: 08:34:21
iteration: 746 | loss: {'ner': 9.226623371002146} | time: 08:34:22
iteration: 747 | loss: {'ner': 3.2437235271391542} | time: 08:34:23
iteration: 748 | loss: {'ner': 3.3125761415230004} | time: 08:34:24
iteration: 749 | loss: {'ner': 4.041095523862184} | ti

iteration: 857 | loss: {'ner': 2.18064356102643} | time: 08:36:02
iteration: 858 | loss: {'ner': 3.958331695529788} | time: 08:36:03
iteration: 859 | loss: {'ner': 1.7490407908028307} | time: 08:36:04
iteration: 860 | loss: {'ner': 2.576889575779945} | time: 08:36:04
iteration: 861 | loss: {'ner': 2.0633954328993513} | time: 08:36:05
iteration: 862 | loss: {'ner': 1.9755531422981396} | time: 08:36:06
iteration: 863 | loss: {'ner': 1.986129828558435} | time: 08:36:07
iteration: 864 | loss: {'ner': 1.892954032791863} | time: 08:36:08
iteration: 865 | loss: {'ner': 1.6669937948211209} | time: 08:36:09
iteration: 866 | loss: {'ner': 3.689458183797956} | time: 08:36:10
iteration: 867 | loss: {'ner': 4.053711850373594} | time: 08:36:11
iteration: 868 | loss: {'ner': 2.719157038166839} | time: 08:36:12
iteration: 869 | loss: {'ner': 1.998945205239486} | time: 08:36:13
iteration: 870 | loss: {'ner': 2.007930421099017} | time: 08:36:13
iteration: 871 | loss: {'ner': 1.7760613617283056} | time: 

iteration: 979 | loss: {'ner': 6.013347827386328} | time: 08:37:55
iteration: 980 | loss: {'ner': 2.0427977402780386} | time: 08:37:56
iteration: 981 | loss: {'ner': 1.4795954351480785} | time: 08:37:57
iteration: 982 | loss: {'ner': 1.9987281024968555} | time: 08:37:58
iteration: 983 | loss: {'ner': 3.2388178029075227} | time: 08:37:59
iteration: 984 | loss: {'ner': 3.0397834522018714} | time: 08:38:00
iteration: 985 | loss: {'ner': 1.9997178595595566} | time: 08:38:01
iteration: 986 | loss: {'ner': 6.6059534991717515} | time: 08:38:02
iteration: 987 | loss: {'ner': 3.9949771543997} | time: 08:38:03
iteration: 988 | loss: {'ner': 2.41321372293294} | time: 08:38:04
iteration: 989 | loss: {'ner': 1.944018070764492} | time: 08:38:05
iteration: 990 | loss: {'ner': 5.658027291939675} | time: 08:38:06
iteration: 991 | loss: {'ner': 2.09504275982728} | time: 08:38:07
iteration: 992 | loss: {'ner': 2.0000110229825707} | time: 08:38:07
iteration: 993 | loss: {'ner': 1.2660340739096227} | time: